# Analysis of Strang-carryover with RK3 for AdImEx Advection
## Plan
- Discretise the advection equation in space using quasi-cubic
- Aapply Strang-carryover adaptively implicitly (AdImEx) with high-order corrections treated explicitly.
- Analyse the stabiltiy of the Strang-carryover scheme

## Discretisation of the Advection Equation
\begin{eqnarray}
\frac{\partial \psi}{\partial t} &=& -u \frac{\partial \psi}{\partial x} \\
\text{Assume } \psi_j &=& e^{ikj\Delta x},\ c = u\Delta t/\Delta x,\ \lambda = u/\Delta x \\
\end{eqnarray}
\begin{eqnarray}
\text{Analytic } \frac{dy}{dt} &=& - i\lambda k\Delta x y \\
\end{eqnarray}
The spatially discretised advection equation can be written as a correction on upwind:
\begin{eqnarray}
\frac{dy}{dt} &=& -\lambda(\mu  + \eta) y\\
\text{where } \mu &=& 1-\cos k\Delta x + i\sin k\Delta x \ \text{ (upwind)}\\
\text{and }   \eta &=& -\mu + \left\{
\sum_\ell w_\ell\  e^{i\ell k\Delta x} - \sum_{\ell-1} w_\ell\  e^{i\ell k\Delta x}
\right\}  \ \text{ (correction)}
\end{eqnarray}

The correction weights for a more general, higher-order discretisation of $\frac{\partial \psi}{\partial x}$:
\begin{eqnarray}
\frac{\partial \psi}{\partial x}_{HOj} &=& \frac{\psi_{j+1/2} - \psi_{j-1/2}}{\Delta x} \\
\text{where } \psi_{j+1/2} &=& \sum_{j+\ell} w_\ell\ \psi_{j+\ell}\\
\implies
\frac{\partial \psi}{\partial x}_{HOj} &=& \sum_{j+\ell} \frac{w_\ell\ \psi_{j+\ell}}{\Delta x}
                                    - \sum_{j+\ell-1} \frac{w_\ell\ \psi_{j+\ell}}{\Delta x}\\
\implies -u\frac{\partial \psi}{\partial x}_{HOj} &=& -\psi_j \lambda \left\{
\sum_\ell w_\ell\  e^{i\ell k\Delta x} - \sum_{\ell-1} w_\ell\  e^{i\ell k\Delta x}
\right\}
\end{eqnarray}


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
#import fluidfoam as foam
from sympy import latex
from sympy import I, E, pi, Q

In [ ]:
# mu and eta for quasi-cubic
kDx = sy.symbols("kΔx", real=True)
mu = 1 - E**(-I*kDx)
eta = -mu

indicies = [-1,0,1]
weights = [sy.Rational(-1,6), sy.Rational(5,6), sy.Rational(1,3)]
for j,w in zip(indicies, weights):
    eta += w*(E**(j*I*kDx) - E**((j-1)*I*kDx))

#print(mu.subs({kDx: pi/3}).evalf())
#print(eta.subs({kDx: pi/3}).evalf())
eta

In [ ]:
# RK3 Butcher Tableau
quarter = sy.Rational(1,4)
sixth = sy.Rational(1,6)
half  = sy.Rational(1,2)
RK3 = [[1,0,0], [quarter,quarter,0], [sixth, sixth, 4*sixth]]
RK1 = [[1]]
RK = RK3

In [ ]:
# Amplification factors for Strang carry-over AdImEx with RK3
c, alpha, beta, gamma = sy.symbols("c, alpha, beta, gamma", real=True, positive=True)
A = sy.Matrix(sy.symarray("A", (len(RK)+2,)))
A[0] = 1 - c*(1-alpha)*beta*mu
for i in range(1,len(RK)+1):
    A[i] = A[0]
    for j in range(0,i):
        A[i] -= c*((1-beta)*mu + gamma*eta)*RK[i-1][j]*A[j]
A[-1] = A[-2]/(1 + c*alpha*beta*mu)

In [ ]:
# Amplification factor for beta = 1-1/c, alpha=1/2, gamma=1
Ag1 = A[-1].subs({alpha:half, beta:1-1/c, gamma:1})
Ag1 = sy.collect(Ag1, c)
Ag1 = sy.simplify(Ag1)
Ag1

In [ ]:
# Amplification for functional forms of alpha, beta and kdx
Aab = A[-1].subs({alpha: 1-1/c, beta: 1-1/c, kDx: pi})
Aab = sy.refine(Aab, Q.positive(c-2))
Aab = sy.refine(Aab, Q.positive(gamma))
Aab = sy.refine(Aab, Q.positive(1-gamma))
g = sy.solve(sy.re(Aab)**2 + sy.im(Aab)**2 -1, gamma)
magSqrA = sy.collect(sy.re(Aab)**2 + sy.im(Aab)**2, c)
magSqrA = sy.simplify(magSqrA)
magSqrA

In [ ]:
# Limiting values of |Aab|
# For large c
magA_cBig = sy.sqrt(125*c**6*gamma**6/26244)
magA_cBig = sy.simplify(magA_cBig)
magA_cBig

In [ ]:
# What is gamma for large c?
g = sy.solve(magA_cBig-1, gamma)
print(g)
g[0].subs({c:1}).evalf()

In [ ]:
# Amplification factor for alpha=0.5, beta=1-1/c
Ai = A[-1].subs({alpha: half, beta: 1-1/c, kDx: pi/2})
Ai = sy.refine(Ai, Q.positive(c-2))
Ai = sy.refine(Ai, Q.positive(gamma))
Ai = sy.refine(Ai, Q.positive(1-gamma))

magSqrAi = sy.collect(sy.re(Ai)**2 + sy.im(Ai)**2, c)
magSqrAi = sy.simplify(magSqrAi)
magSqrAi

cMax = sy.solve(magSqrAi.subs({gamma:1})-1, c)
for ci in cMax:
    print(ci.evalf())

In [ ]:
def magA(cs, gs):
    try:
        cs, gs = np.meshgrid(cs, gs)
    except:
        pass
    return sy.lambdify([c, gamma], magSqrA**.5, 'numpy')(cs, gs)

cs = np.linspace(2, 20, 36)
gs = np.linspace(0, 1, 11)
Ag = magA(cs, gs)

In [ ]:
g = sy.solve(magSqrA -1, gamma)
#print(g)

# Print solutions using magSqrA and magA
for ga in g:
    ga = sy.simplify(ga)
    ga = sy.collect(ga, c)
    ga = sy.simplify(ga)
    #print('Solution :', ga)
    Co = 20
    Ga = ga.subs({c: Co})
    if sy.im(Ga) == 0:
        if Ga > 0. and Ga < 1.:
            #print('gamma =', Ga.evalf(), 'c =', Co, 'magSqrA =', magSqrA.subs({gamma: Ga, c: Co}).evalf(),
            #      'magA =', magA(Co, Ga.evalf()))
            #print('g(c) = ',ga)
            gByc = sy.lambdify(c, ga, 'numpy')

In [ ]:
plt.contourf(cs, gs, Ag)
plt.colorbar()
plt.contour(cs, gs, Ag, [1,100], colors='w')
plt.plot(cs, 6.5/np.maximum(1, cs+4), 'k--', label='6.5/(c+4)')
plt.plot(cs, 6.5/np.maximum(1, cs+5), 'r--', label='6.5/(c+5)')
plt.plot(cs, 6.5/np.maximum(1, cs+3.5), 'y--', label='6.5/(c+3.5)')
plt.plot(cs, 5/np.maximum(1, cs+4), 'w--', label='5/(c+4)')
plt.plot(cs, 2/np.maximum(2, cs), 'g--', label='2/c')
#plt.plot(cs, gByc(cs), label='gByc')
plt.legend()
plt.xlabel('c')
plt.ylabel(r'$\gamma$')
plt.ylim([0,1])
plt.title(r'|A| for $\alpha=1-1/c$, $\beta=1-1/c$')
plt.savefig('plots/AadImExgamma.pdf')
plt.show()

In [ ]:
# |A| for beta=1-1/c, alpha=1/2, gamma=1
def magA(cs, kdxs):
    try:
        cs, kdxs = np.meshgrid(cs, kdxs)
    except:
        pass
    return sy.lambdify([c, kDx], sy.Abs(Ag1), 'numpy')(cs, kdxs)

cs = np.linspace(1, 5, 21)
kdxs = np.linspace(0, np.pi, 21)
Ai = magA(cs, kdxs)

plt.contourf(cs, kdxs, Ai)
plt.colorbar()
plt.contour(cs, kdxs, Ai, [-1,1], colors='w')
plt.xlabel('c')
plt.ylabel(r'$k\Delta x$')
plt.title(r'|A| for $\alpha=1/2$, $\beta=1-1/c$, $\gamma=1$')
plt.show()